<a href="https://colab.research.google.com/github/mertserbes/wargame3d/blob/main/Wargame3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>